In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

import torch
import torchvision
import cv2
import pydicom
import random
import matplotlib.pyplot as plt
import math
import argparse
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

In [ ]:
def label_to_name(id):
    id = int(id)
    id = id-1
    if id == 0:
        return "Aortic enlargement"
    if id == 1:
        return "Atelectasis"
    if id == 2:
        return "Calcification"
    if id == 3:
        return "Cardiomegaly"
    if id == 4:
        return "Consolidation"
    if id == 5:
        return "ILD"
    if id == 6:
        return "Infiltration"
    if id == 7:
        return "Lung Opacity"
    if id == 8:
        return "Nodule/Mass"
    if id == 9:
        return "Other lesion"
    if id == 10:
        return "Pleural effusion"
    if id == 11:
        return "Pleural thickening"
    if id == 12:
        return "Pneumothorax"
    if id == 13:
        return "Pulmonary fibrosis"
    else:
        return str(id)

In [ ]:

DIR_INPUT = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [ ]:
train_df = pd.read_csv(f'{DIR_INPUT}/train.csv')
train_df.fillna(0, inplace=True)
train_df.loc[train_df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

print(f"df Shape: {train_df.shape}")
print("No Of Classes: ", train_df["class_id"].nunique())
train_df.sort_values(by='image_id').head(10)

In [ ]:
image_ids = train_df['image_id'].sample(frac = 1).unique()
validation_ids = image_ids[-3000:]# Tran and Validation Split 
training_ids = image_ids[:-3000]


validation_df = train_df[train_df['image_id'].isin(validation_ids)]
training_df= train_df[train_df['image_id'].isin(training_ids)]


training_ids.shape


In [ ]:
def define_box(image, boxes, IMG_SIZE):
    boxes['x_min'] = ((boxes['x_min']/ np.shape(image)[0])* IMG_SIZE)
    boxes['y_min'] = ((boxes['y_min']/ np.shape(image)[1])* IMG_SIZE)
    
    boxes['x_max'] = ((boxes['x_max']/ np.shape(image)[0])* IMG_SIZE)
    boxes['y_max'] = ((boxes['y_max']/ np.shape(image)[1])* IMG_SIZE)
    return boxes

In [ ]:
def image_pad(image):
    row_size = np.shape(image)[0]
    col_size = np.shape(image)[1]
    image = np.pad(image, ((math.floor((4000-row_size)/2), math.ceil((4000-row_size)/2)), 
                           (math.floor((4000-col_size)/2), math.ceil((4000-col_size)/2))),
                           'constant', constant_values=(0, 0))
    return image

In [ ]:
class VinBigDataset(Dataset): #Class to load Training Data
    
    def __init__(self, dataframe, image_dir, transforms=None,stat = 'Train'):
        super().__init__()
        
        self.image_ids = dataframe["image_id"].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.stat = stat
        
    def __getitem__(self, index):
        IMG_SIZE = 512  
        image_id = self.image_ids[index]
        records = self.df[(self.df['image_id'] == image_id)]
        records = records.reset_index(drop=True)

        dicom = pydicom.dcmread(f"{self.image_dir}/{image_id}.dicom")

        image = dicom.pixel_array
      #  new_image = image
        new_image = cv2.resize(image, (IMG_SIZE,IMG_SIZE))# norm the value dont div on 255 
       # new_image = np.reshape(new_image, (IMG_SIZE, IMG_SIZE))
        # new_image = np.asarray(new_image)
        """
        image = image_pad(image)
        #'MONOCHROME1' - White and black are inverted 
        if "PhotometricInterpretation" in dicom:
            if dicom.PhotometricInterpretation == "MONOCHROME1":
                #Inverting Black and White
                image = np.amax(image) - image
        #dicom images are stored in certain way. In order to retrieve the image I, we tranform with image I= RescaleSlope* image+RescaleIntercept
        intercept = dicom.RescaleIntercept if "RescaleIntercept" in dicom else 0.0
        slope = dicom.RescaleSlope if "RescaleSlope" in dicom else 1.0

        image = slope * image
        image += intercept

        #Rescaling the image
        image = image - image.min()
        image = image / image.max()
        image = image * 255.0
        #image = image.transpose(1,2,0)
        """
        if records.loc[0, "class_id"] == 0:
            records = records.loc[[0], :]

        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']]
        boxes = define_box(image, boxes, IMG_SIZE).values
        labels = torch.tensor(records["class_id"].values, dtype=torch.int64)


        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            sample = {
                'image': new_image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            new_image = torch.tensor(sample['image'])
            
            target['boxes'] = torch.tensor(sample['bboxes'])

        if target["boxes"].shape[0] == 0:
            # Albumentation cuts the target (class 14, 1x1px in the corner)
            target["boxes"] = torch.from_numpy(np.array([[0.0, 0.0, 1.0, 1.0]]))
            target["labels"] = torch.tensor([0], dtype=torch.int64)

        return new_image, target, image_ids
    
    def __len__(self):
        return self.image_ids.shape[0]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = VinBigDataset(training_df, DIR_TRAIN)
valid_dataset = VinBigDataset(validation_df, DIR_TRAIN)


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()
# Create train and validate data loader
train_data_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
# Train dataset sample
images, targets, image_ids = next(iter(train_data_loader))
images = list(image for image in images)
image_array = np.asarray(images)

targets = [{k: v for k, v in t.items()} for t in targets]

for number in random.sample([1,2,3,6,5,4],6):
    boxes = targets[number]['boxes'].astype(np.int32)
    img = images[number]
    
    labels= targets[number]['labels'].numpy().astype(np.int32)
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for i in range(len(boxes)):
        img = cv2.rectangle(img,(boxes[i][0],boxes[i][1]),(boxes[i][2],boxes[i][3]),(255,0,0),2)
        #print(le.inverse_transform([labels[i]-1])[0])
        #print(label_to_name(labels[i]), (int(boxes[i][0]), int(boxes[i][1])))
        img = cv2.putText(img, label_to_name(labels[i]), (int(boxes[i][0]), int(boxes[i][1])), cv2.FONT_HERSHEY_TRIPLEX,1, (255,0,0), 2, cv2.LINE_AA)

    ax.set_axis_off()
    ax.imshow(img)

In [ ]:
class CNNEncoderSimple(nn.Module):
    def __init__(self):
        super(CNNEncoderSimple, self).__init__()
        # input_dim  # b, 1, 1024, 1024
        #layer 1
        self.conv1 = nn.Conv2d(1, 8, 3, stride=1, padding=1)  # b, 8, 512,512
        self.pool1 = nn.MaxPool2d(2, stride=2)  # b, 8, 256,256
        self.conv1_bn = nn.BatchNorm2d(8)
        #layer 2
        self.conv2 = nn.Conv2d(8, 16, 3, stride=1, padding=1)  # b, 16, 256, 256
        self.pool2 = nn.MaxPool2d(2, stride=2) #b, 16, 128, 128
        self.conv2_bn = nn.BatchNorm2d(16)
        #layer 3
     #   self.conv3 = nn.Conv2d(16, 32, 3, stride=1, padding=1)  # b, 32, 256, 256
     #   self.pool3 = nn.MaxPool2d(2, stride=2) #b, 32, 128, 128
     #   self.conv3_bn = nn.BatchNorm2d(32)
        
        #layer 4
        self.conv4 = nn.Conv2d(16, 8, 1, stride=1, padding=0)  #b, 32, 128, 128
        self.conv4_bn = nn.BatchNorm2d(8)
        #layer 5
        self.conv5 = nn.Conv2d(8, 1, 1, stride=1, padding=0)  #b, 8, 128, 128
        self.conv5_bn = nn.BatchNorm2d(1)
        #layer 6
        self.conv6 = nn.Conv2d(1, 1, 2, stride=2, padding=0)  #b, 1, 64, 64
        
        #self.fc1 = nn.Linear(32 * 12*12*12, 16 * 12*12*12)  # b, 55296 to 27648
        #self.fc1_bn = nn.BatchNorm1d(16 * 12*12*12)
        #self.fc2 = nn.Linear(16 * 12*12*12, 12*12*12)  # b, 27648 to 1728
    
    def forward(self, x):
        #print("0 : x.", x.size())
   #     x = F.leaky_relu_(self.conv1(x))
     #   print("1:",x.size())
    #    x = self.conv1_bn(self.pool1(x)) 
        print("2:",x.size())
        x = (F.leaky_relu_(self.conv2(x)))
        x = self.conv2_bn(self.pool2(x)) 
        print("3:",x.size())
        x = (F.leaky_relu_(self.conv3(x)))
        x = self.conv3_bn(self.pool3(x)) 
        x = (F.leaky_relu_(self.conv4(x)))
        x = self.conv4_bn(x)
        print("4:",x.size())
        x = (F.leaky_relu_(self.conv5(x)))
        x = self.conv5_bn(x)
        print("5:",x.size())
        x = self.conv6(x)
        print("6:",x.size())
        x = x.reshape(x.shape[0], -1)
        #print("4:",x.size())
        #x = self.fc1_bn(F.leaky_relu_(self.fc1(x)))
        #print("5:",x.size())
        #x = self.fc2(x)
        #print("6:",x.size())
        #exit()
        return x    



class CNNDecoderSimple(nn.Module):
    def __init__(self):
        super(CNNDecoderSimple, self).__init__()
        # Fully Connected Layers
        self.fc1 = nn.Linear(64*64, 8*64*64)  # b, 4096 to 32,768
        self.fc1_bn = nn.BatchNorm1d(8*64*64)

        self.fc2 = nn.Linear(8*64*64,8 * 128*128)  # b, 32,768 to 131,072
        self.fc2_bn = nn.BatchNorm1d(8*128*128)
 
        self.fc3 = nn.Linear(8*128*128,32 * 128*128)  # b, 131,072 to 
        self.fc3_bn = nn.BatchNorm1d(16*128*128)

      #  self.conv1 = nn.ConvTranspose2d(32, 16, 3, stride=1, padding=1)  # b, 16, 20,20,20
       # self.conv1_bn = nn.BatchNorm2d(16)
        
        self.conv1 = nn.ConvTranspose2d(16, 8, 3, stride=1, padding=1)  # b, 16, 20,20,20
        self.conv1_bn = nn.BatchNorm2d(8)
        
        self.conv2 = nn.ConvTranspose2d(8, 1, 3, stride=1, padding=1)  # b, 16, 40,40,40
        

        
    
    def forward(self, x):
        x = self.fc1_bn(F.leaky_relu_(self.fc1(x)))
        x = self.fc2_bn(F.leaky_relu_(self.fc2(x)))
        x = self.fc3_bn(F.leaky_relu_(self.fc3(x)))
        #print("After fc:",x.size())
        x = x.reshape(x.shape[0], 16,128, 128)
        #print("Reshape :",x.size())
        x = self.conv1_bn(F.leaky_relu_(self.conv1(x)))
     #   x = self.conv2_bn(F.leaky_relu_(self.conv2(x)))

        #print("Conv1 :", x.size())
        x = nn.Tanh()(self.conv2(x))
        #print("Conv2 :", x.size())
        #exit()
        
        
        return x

In [ ]:
def run_epoch(epoch, dataloader,encoder, decoder, optimizer, criterion, is_eval=False):
    losses = []
    if is_eval:
        encoder.eval()
    #    decoder.eval()
    else:
        encoder.train()
    #    decoder.train()
    for i,data in enumerate(dataloader):
        if i % (10000 // 16) == 0:
            print(f"{i}/{len(dataloader)}")
        
        image  = data["image"]
        #image = image*2 - 1
        image = image.to(device="cuda:0",dtype=torch.float)
        
        # ===================forward=====================
        h = encoder(image)
        #output = decoder(h)
        #loss = criterion(output, image)
        #losses.append(loss.detach())
        # ===================backward====================
        #if not is_eval:
         #   optimizer.zero_grad()
          #  loss.backward()
           # optimizer.step()
        
    
    #loss = torch.FloatTensor(losses).mean()
    return None

In [ ]:
def pretrainer():
    batch_size = 2
    #dataset_train = load_data(f'{dataset_path}/training_images/', target_transform=transform)
   # dataset_valid = load_data(f'{dataset_path}/validation_images/', target_transform=transform)
    train_dataset = VinBigDataset(training_df, DIR_TRAIN)
    dataloader_train = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, collate_fn=collate_fn, shuffle=True)
    #print(dataloader_train)
    valid_dataset = VinBigDataset(validation_df, DIR_TRAIN)
    dataloader_valid = DataLoader(valid_dataset, batch_size=batch_size, num_workers=4, collate_fn=collate_fn, shuffle=False)
    #print(len(dataloader_train))
    
    encoder = CNNEncoderSimple()
    decoder = CNNDecoderSimple()

    epochs = 1
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=learning_rate)
    criterion = nn.MSELoss(reduction='mean')
    
    for epoch in range(epochs):
        epoch = epoch + 1
        print("epoch = ", epoch)
        if ((epoch+1)%10 == 0):
            learning_rate = learning_rate/10.0
            optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=learning_rate)
     #   loss_train = run_epoch(epoch, dataloader_train, encoder, decoder, optimizer, criterion, is_eval=False)
        loss_valid = run_epoch(epoch, dataloader_valid, encoder, decoder, optimizer, criterion, is_eval=True)
        
    return None

In [ ]:
pretrainer()